In [35]:
# imports
import numpy as np
import pandas as pd
import seaborn as sns
import re
from scipy import stats
from pathlib import Path

# directory and file paths
data_dir = Path("../data/")
raw_data_dir = data_dir / "raw"
processed_data_dir = data_dir / "processed"
interim_data_dir = data_dir / "interim"

raw_rental_vacancy_file = raw_data_dir / "rental_vacancy_rates_75.csv"
raw_homeowner_vacancy_file = raw_data_dir / "homeowner_vacancy_rates_75.csv"
raw_homeownership_file = raw_data_dir / "homeownership_rates_75.csv"
raw_covid_cases_file = raw_data_dir / "COVID-19_Case_Surveillance_Public_Use_Data.csv"
raw_geo_covid_cases_file = raw_data_dir / "COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv"

interim_rental_vacancy_file = interim_data_dir / "rental_vacancy_rates_75.csv"
interim_homeowner_vacancy_file = interim_data_dir / "homeowner_vacancy_rates_75.csv"
interim_homeownership_file = interim_data_dir / "homeownership_rates_75.csv"

# Housing Vacancies and Homeownership (Census Data)

In [23]:
# Since the datasets are stored similarly to pivot tables,
# they need to be flattened in order to manipulate
def flatten_df(df):
    years = ["2020","2019","2018","2017","2016","2015"]
    quarters = ["Q1 ","Q2 ","Q3 ","Q4 "]
    moes = ['Margin of Error1', 'Margin of Error1.1', 'Margin of Error1.2',
       'Margin of Error1.3', 'Margin of Error1.4', 'Margin of Error1.5',
       'Margin of Error1.6', 'Margin of Error1.7', 'Margin of Error1.8',
       'Margin of Error1.9', 'Margin of Error1.10', 'Margin of Error1.11',
       'Margin of Error1.12', 'Margin of Error1.13', 'Margin of Error1.14',
       'Margin of Error1.15', 'Margin of Error2', 'Margin of Error1.16',
       'Margin of Error1.17', 'Margin of Error1.18', 'Margin of Error3',
       'Margin of Error3.1', 'Margin of Error3.2', 'Margin of Error3.3']
    data = []
    for index, row in df.iterrows():
        count = -1
        for j,year in enumerate(years):
            for k,quarter in enumerate(quarters):
                count += 1
                data.append([row["Metropolitan Statistical Area"],
                                quarter,year,
                                row[quarter + year],
                                row[moes[count]]])
    return pd.DataFrame(data=data,columns=['MSA','quarter','year','vacancy rate','margin of error'])

## Rental Vacancy Rates for the 75 Largest MSAs (2015 - 2020)

In [24]:
# import and clean dataset
rental_vacancy_df = pd.read_csv(interim_rental_vacancy_file)

rental_vacancy_df["Metropolitan Statistical Area"] = rental_vacancy_df["Metropolitan Statistical Area"
                                                                      ].str.replace('[\.]+','')
rental_vacancy_df["Metropolitan Statistical Area"] = rental_vacancy_df["Metropolitan Statistical Area"
                                                                      ].str.replace('[\…]+','')
rental_vacancy_df = rental_vacancy_df.replace('(z)', np.NaN, regex=False)
rental_vacancy_df = rental_vacancy_df.rename(columns=lambda x: re.sub('First[\s]*Quarter','Q1',x))
rental_vacancy_df = rental_vacancy_df.rename(columns=lambda x: re.sub('Second[\s]*Quarter','Q2',x))
rental_vacancy_df = rental_vacancy_df.rename(columns=lambda x: re.sub('Third[\s]*Quarter','Q3',x))
rental_vacancy_df = rental_vacancy_df.rename(columns=lambda x: re.sub('Fourth[\s]*Quarter','Q4',x))
rental_vacancy_df = rental_vacancy_df.rename(columns=lambda x: re.sub('[\s]+',' ',x))

rental_vacancy_df.head()

,Metropolitan Statistical Area,Q1 2020,Margin of Error1,Q2 2020,Margin of Error1.1,Q3 2020,Margin of Error1.2,Q4 2020,Margin of Error1.3,Q1 2019,...,Q4 2016,Margin of Error1.18,Q1 2015,Margin of Error3,Q2 2015,Margin of Error3.1,Q3 2015,Margin of Error3.2,Q4 2015,Margin of Error3.3
0,"Akron, OH",10.2,9.2,8.1,8.5,3.3,5.3,3.6,5.0,2.0,...,5.2,8.0,3.5,5.1,14.6,9.7,19.9,11.9,10.3,8.9
1,"Albany-Schenectady-Troy, NY",7.9,6.7,6.7,5.6,10.1,7.1,15.3,10.2,7.9,...,3.6,4.6,9.6,6.8,7.9,5.7,5.6,5.1,3.0,3.9
2,"Albuquerque, NM",4.3,2.8,6.9,4,6.8,3.4,4.1,2.5,7.1,...,11.6,4.3,7.3,3.3,7.7,3.3,7.4,3.6,6.4,3.5
3,"Allentown-Bethlehem-Easton, PA-NJ",4.5,5.9,5.6,6.6,2.9,4.2,2.5,4.9,7.7,...,4.6,6.7,3.5,4.6,3.4,5.1,6.0,6.3,2.4,3.7
4,"Atlanta-Sandy Springs-Roswell, GA1",6.9,2.5,5.1,2.3,5.8,2.4,7.7,2.7,7.8,...,6.4,2.6,9.5,2.5,8.4,2.3,7.9,2.3,6.8,2.3


In [25]:
# convert quasi pivot table into flat dataframe
rv_flat_df = flatten_df(rental_vacancy_df)
rv_flat_df = rv_flat_df.rename(columns={"vacancy rate":"rental vacancy rate",
                                        "margin of error":"rental vacancy rate margin of error"})
rv_flat_df.head()

,MSA,quarter,year,rental vacancy rate,rental vacancy rate margin of error
0,"Akron, OH",Q1,2020,10.2,9.2
1,"Akron, OH",Q2,2020,8.1,8.5
2,"Akron, OH",Q3,2020,3.3,5.3
3,"Akron, OH",Q4,2020,3.6,5
4,"Akron, OH",Q1,2019,2,4.1


## Homeowner Vacancy Rates for the 75 Largest MSAs (2015 - 2020)

In [26]:
# import and clean dataset
homeowner_vacancy_df = pd.read_csv(interim_homeowner_vacancy_file)
homeowner_vacancy_df["Metropolitan Statistical Area"] = homeowner_vacancy_df["Metropolitan Statistical Area"
                                                                      ].str.replace('[\.]+','')
homeowner_vacancy_df["Metropolitan Statistical Area"] = homeowner_vacancy_df["Metropolitan Statistical Area"
                                                                      ].str.replace('[\…]+','')
homeowner_vacancy_df = homeowner_vacancy_df.replace('(z)', np.NaN, regex=False)
homeowner_vacancy_df = homeowner_vacancy_df.rename(columns=lambda x: re.sub('First[\s]*Quarter','Q1',x))
homeowner_vacancy_df = homeowner_vacancy_df.rename(columns=lambda x: re.sub('Second[\s]*Quarter','Q2',x))
homeowner_vacancy_df = homeowner_vacancy_df.rename(columns=lambda x: re.sub('Third[\s]*Quarter','Q3',x))
homeowner_vacancy_df = homeowner_vacancy_df.rename(columns=lambda x: re.sub('Fourth[\s]*Quarter','Q4',x))
homeowner_vacancy_df = homeowner_vacancy_df.rename(columns=lambda x: re.sub('[\s]+',' ',x))
homeowner_vacancy_df.head()

,Metropolitan Statistical Area,Q1 2020,Margin of Error1,Q2 2020,Margin of Error1.1,Q3 2020,Margin of Error1.2,Q4 2020,Margin of Error1.3,Q1 2019,...,Q4 2016,Margin of Error1.18,Q1 2015,Margin of Error3,Q2 2015,Margin of Error3.1,Q3 2015,Margin of Error3.2,Q4 2015,Margin of Error3.3
0,"Akron, OH",0.5,1.4,0.5,1.4,0.5,1.5,NaN,NaN,0.0,...,0.5,1.6,0.0,NaN,0.5,1.3,0.8,1.5,0.5,1.3
1,"Albany-Schenectady-Troy, NY",0.9,1.9,0.9,1.9,5.3,4.2,4.9,4,1.6,...,2.2,2.9,1.3,2,2.8,2.7,1.8,2.1,1.1,1.7
2,"Albuquerque, NM",2,1.3,1.6,1.1,1,1,0.7,0.8,1.9,...,1.5,1.3,2.5,1.6,3.9,2,2.5,1.6,2.1,1.5
3,"Allentown-Bethlehem-Easton, PA-NJ",0.6,1.4,0.5,1.4,0.5,1.4,1.3,2.2,1.2,...,1.8,2.9,0.5,1.2,2.5,2.7,2.1,2.5,2.0,2.6
4,"Atlanta-Sandy Springs-Roswell, GA1",1.2,0.8,1,0.7,0.3,0.4,0.8,0.7,0.9,...,1.5,1,2.9,1.2,2.4,1.1,1.8,0.9,1.7,0.9


In [27]:
# convert quasi pivot table into flat dataframe
hv_flat_df = flatten_df(homeowner_vacancy_df)
hv_flat_df = hv_flat_df.rename(columns={"vacancy rate":"homeowner vacancy rate",
                                        "margin of error":"homeowner vacancy rate margin of error"})
hv_flat_df.head()

,MSA,quarter,year,homeowner vacancy rate,homeowner vacancy rate margin of error
0,"Akron, OH",Q1,2020,0.5,1.4
1,"Akron, OH",Q2,2020,0.5,1.4
2,"Akron, OH",Q3,2020,0.5,1.5
3,"Akron, OH",Q4,2020,NaN,NaN
4,"Akron, OH",Q1,2019,0,NaN


## Homeownership Rates for the 75 Largest MSAs (2015 - 2020)

In [28]:
# import and clean dataset
homeownership_df = pd.read_csv(interim_homeownership_file)
homeownership_df["Metropolitan Statistical Area"] = homeownership_df["Metropolitan Statistical Area"
                                                                      ].str.replace('[\.]+','')
homeownership_df["Metropolitan Statistical Area"] = homeownership_df["Metropolitan Statistical Area"
                                                                      ].str.replace('[\…]+','')
homeownership_df = homeownership_df.replace('(z)', np.NaN, regex=False)
homeownership_df = homeownership_df.rename(columns=lambda x: re.sub('First[\s]*Quarter','Q1',x))
homeownership_df = homeownership_df.rename(columns=lambda x: re.sub('Second[\s]*Quarter','Q2',x))
homeownership_df = homeownership_df.rename(columns=lambda x: re.sub('Third[\s]*Quarter','Q3',x))
homeownership_df = homeownership_df.rename(columns=lambda x: re.sub('Fourth[\s]*Quarter','Q4',x))
homeownership_df = homeownership_df.rename(columns=lambda x: re.sub('[\s]+',' ',x))
homeownership_df.head()

,Metropolitan Statistical Area,Q1 2020,Margin of Error1,Q2 2020,Margin of Error1.1,Q3 2020,Margin of Error1.2,Q4 2020,Margin of Error1.3,Q1 2019,...,Q4 2016,Margin of Error1.18,Q1 2015,Margin of Error3,Q2 2015,Margin of Error3.1,Q3 2015,Margin of Error3.2,Q4 2015,Margin of Error3.3
0,"Akron, OH",71.9,9.7,71.7,9.9,68.5,10.1,66.4,9.6,68.3,...,73.2,10.2,68.6,8.7,73.9,8.0,79.9,7.3,73.4,8.4
1,"Albany-Schenectady-Troy, NY",62.9,9.7,57.7,9.6,62.3,9.5,73.1,9.2,60.9,...,60.9,9.2,65.9,8.2,63.5,7.7,66.6,7.5,67.5,7.4
2,"Albuquerque, NM",70.9,4.4,77.2,4.1,66.9,4.7,63.1,4.7,72.1,...,64.7,4.8,63.8,4.6,60.5,4.8,65.9,4.7,67.4,4.7
3,"Allentown-Bethlehem-Easton, PA-NJ",73.1,8.7,69.3,9.5,61.4,9.6,71.9,9.6,70.5,...,68.4,10.2,69.2,7.7,72.7,8.0,71.5,8.0,63.5,8.6
4,"Atlanta-Sandy Springs-Roswell, GA1",63.7,3.8,68.3,3.6,66.6,3.6,67.0,3.7,62.6,...,61.6,3.9,60.5,3.3,59.9,3.2,60.9,3.2,65.5,3.1


In [29]:
# convert quasi pivot table into flat dataframe
homeownership_flat_df = flatten_df(homeownership_df)
homeownership_flat_df = homeownership_flat_df.rename(columns={"vacancy rate":"homeownership rate",
                                                              "margin of error":"homeownership rate margin of error"})
homeownership_flat_df.head()

,MSA,quarter,year,homeownership rate,homeownership rate margin of error
0,"Akron, OH",Q1,2020,71.9,9.7
1,"Akron, OH",Q2,2020,71.7,9.9
2,"Akron, OH",Q3,2020,68.5,10.1
3,"Akron, OH",Q4,2020,66.4,9.6
4,"Akron, OH",Q1,2019,68.3,9.9


In [36]:
housing_df = rv_flat_df.merge(hv_flat_df)
housing_df = housing_df.merge(homeownership_flat_df)
housing_csv = housing_df.to_csv(processed_data_dir/"housing_csv.csv",index=False)
housing_df.head()

,MSA,quarter,year,rental vacancy rate,rental vacancy rate margin of error,homeowner vacancy rate,homeowner vacancy rate margin of error,homeownership rate,homeownership rate margin of error
0,"Akron, OH",Q1,2020,10.2,9.2,0.5,1.4,71.9,9.7
1,"Akron, OH",Q2,2020,8.1,8.5,0.5,1.4,71.7,9.9
2,"Akron, OH",Q3,2020,3.3,5.3,0.5,1.5,68.5,10.1
3,"Akron, OH",Q4,2020,3.6,5,NaN,NaN,66.4,9.6
4,"Akron, OH",Q1,2019,2,4.1,0,NaN,68.3,9.9


# COVID-19 Case Surveillance Public Use Data (CDC Data)

In [31]:
# import dataset
covid_cases_df = pd.read_csv(raw_covid_cases_file)

covid_cases_df.shape[0]

KeyboardInterrupt: 

In [ ]:
geo_covid_cases_df = pd.read_csv(raw_geo_covid_cases_file)
geo_covid_cases_df.head()